In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Load the combined datasets for analysis
dot1 = pd.read_csv('worked_data/dot1_all.csv')
dot2 = pd.read_csv('worked_data/dot2_all.csv')
dot3 = pd.read_csv('worked_data/dot3_all.csv')

# Standardize column names for consistency across datasets
dot1.columns = [c.strip().upper() for c in dot1.columns]
dot2.columns = [c.strip().upper() for c in dot2.columns]
dot3.columns = [c.strip().upper() for c in dot3.columns]

# Preview the first few rows of each dataset to understand structure and content
print("dot1 columns:", dot1.columns.tolist())
display(dot1.head(3))
print("dot2 columns:", dot2.columns.tolist())
display(dot2.head(3))
print("dot3 columns:", dot3.columns.tolist())
display(dot3.head(3))

dot1 columns: ['TRDTYPE', 'USASTATE', 'DEPE', 'DISAGMOT', 'MEXSTATE', 'CANPROV', 'COUNTRY', 'VALUE', 'SHIPWT', 'FREIGHT_CHARGES', 'DF', 'CONTCODE', 'MONTH', 'YEAR', 'SOURCE_FILE']


,TRDTYPE,USASTATE,DEPE,DISAGMOT,MEXSTATE,CANPROV,COUNTRY,VALUE,SHIPWT,FREIGHT_CHARGES,DF,CONTCODE,MONTH,YEAR,SOURCE_FILE
0,1,AK,0712,5,NaN,XA,1220,7204,0,248,1.0,X,2,2023,dot1_0223.csv
1,1,AK,0712,5,NaN,XQ,1220,175723,0,1740,2.0,X,2,2023,dot1_0223.csv
2,1,AK,18XX,1,XX,NaN,2010,10244,1512,0,1.0,1,2,2023,dot1_0223.csv


dot2 columns: ['TRDTYPE', 'USASTATE', 'COMMODITY2', 'DISAGMOT', 'MEXSTATE', 'CANPROV', 'COUNTRY', 'VALUE', 'SHIPWT', 'FREIGHT_CHARGES', 'DF', 'CONTCODE', 'MONTH', 'YEAR', 'SOURCE_FILE']


,TRDTYPE,USASTATE,COMMODITY2,DISAGMOT,MEXSTATE,CANPROV,COUNTRY,VALUE,SHIPWT,FREIGHT_CHARGES,DF,CONTCODE,MONTH,YEAR,SOURCE_FILE
0,1,AK,2,5,TM,NaN,2010,5948,0,0,1.0,0,2,2023,dot2_0223.csv
1,1,AK,3,5,NaN,XC,1220,456681,0,9302,1.0,X,2,2023,dot2_0223.csv
2,1,AK,7,5,NaN,XC,1220,73470,0,2832,2.0,X,2,2023,dot2_0223.csv


dot3 columns: ['TRDTYPE', 'DEPE', 'COMMODITY2', 'DISAGMOT', 'COUNTRY', 'VALUE', 'SHIPWT', 'FREIGHT_CHARGES', 'DF', 'CONTCODE', 'MONTH', 'YEAR', 'SOURCE_FILE']


,TRDTYPE,DEPE,COMMODITY2,DISAGMOT,COUNTRY,VALUE,SHIPWT,FREIGHT_CHARGES,DF,CONTCODE,MONTH,YEAR,SOURCE_FILE
0,1,0101,19,5,1220,7790,0,153,2.0,X,2,2023,dot3_0223.csv
1,1,0101,39,5,1220,5080,0,133,1.0,X,2,2023,dot3_0223.csv
2,1,0101,42,5,1220,8793,0,44,1.0,X,2,2023,dot3_0223.csv


In [ ]:
# Analyze the distribution of shipments by transportation mode (DISAGMOT)
mode_counts = dot1['DISAGMOT'].value_counts().sort_values(ascending=False)
print("Top transportation modes (by DISAGMOT code):")
display(mode_counts)

The following stacked bar chart visualizes how freight movement is distributed across US states and transportation modes.

In [ ]:
# Visualize freight movement by US state and mode
state_mode = dot1.groupby(['USASTATE', 'DISAGMOT']).size().unstack(fill_value=0)
state_mode.plot(kind='bar', stacked=True, title='Freight Movement by US State and Mode', figsize=(16,6))
plt.ylabel('Number of Shipments')
plt.xlabel('US State')
plt.legend(title='Mode (DISAGMOT)', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

The following time series plot shows how the use of different transportation modes changes over time.

In [ ]:
# Visualize temporal trends in freight movement by mode
dot1['PERIOD'] = dot1['YEAR'].astype(str) + '-' + dot1['MONTH'].astype(str).str.zfill(2)
time_mode = dot1.groupby(['PERIOD', 'DISAGMOT']).size().unstack(fill_value=0)
time_mode.plot(title='Freight Movement Over Time by Mode')
plt.ylabel('Number of Shipments')
plt.xlabel('Period (Year-Month)')
plt.xticks(rotation=90)
plt.legend(title='Mode (DISAGMOT)', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
# Convert relevant columns to numeric for calculations
dot2['FREIGHT_CHARGES'] = pd.to_numeric(dot2['FREIGHT_CHARGES'], errors='coerce')
dot2['SHIPWT'] = pd.to_numeric(dot2['SHIPWT'], errors='coerce')

# Calculate cost per weight for each shipment
dot2['COST_PER_WEIGHT'] = dot2['FREIGHT_CHARGES'] / dot2['SHIPWT'].replace(0, np.nan)

The following bar chart shows the average cost per weight for each transportation mode.

In [ ]:
# Visualize average cost per weight by transportation mode
mode_ineff = dot2.groupby('DISAGMOT')['COST_PER_WEIGHT'].mean().sort_values(ascending=False)
mode_ineff.plot(kind='bar', title='Average Cost per Weight by Mode')
plt.ylabel('Cost per Weight')
plt.xlabel('Mode (DISAGMOT)')
plt.tight_layout()
plt.show()

The table below lists the top 3 most inefficient mode-route combinations, based on average cost per weight.

In [ ]:
# Identify the top 3 most inefficient mode-route combinations
dot2['ROUTE'] = dot2['USASTATE'].astype(str) + '-' + dot2['MEXSTATE'].astype(str) + '-' + dot2['CANPROV'].astype(str)
route_ineff = dot2.groupby(['DISAGMOT', 'ROUTE'])['COST_PER_WEIGHT'].mean().sort_values(ascending=False)
display(route_ineff.head(3))

In [ ]:
# Aggregate total weight moved by mode and US state
env_impact = dot1.groupby(['DISAGMOT', 'USASTATE'])['SHIPWT'].sum().unstack(fill_value=0)
env_impact.plot(kind='bar', stacked=True, title='Total Weight by Mode and US State')
plt.ylabel('Total Weight Shipped')
plt.xlabel('Mode (DISAGMOT)')
plt.legend(title='US State', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
# Summarize total weight shipped by country and mode
cross_border = dot3.groupby(['COUNTRY', 'DISAGMOT'])['SHIPWT'].sum().unstack(fill_value=0)
cross_border.plot(kind='bar', stacked=True, title='Cross-Border Freight by Country and Mode')
plt.ylabel('Total Weight Shipped')
plt.xlabel('Country')
plt.legend(title='Mode (DISAGMOT)', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

The following table lists the most used modes for cross-border freight, by total weight.

In [ ]:
# List most used modes for cross-border freight
mode_cross = dot3.groupby('DISAGMOT')['SHIPWT'].sum().sort_values(ascending=False)
display(mode_cross)

In [ ]:
# Analyze seasonal trends in freight movement by mode
seasonal = dot1.groupby(['YEAR', 'MONTH', 'DISAGMOT'])['SHIPWT'].sum().reset_index()
seasonal['PERIOD'] = seasonal['YEAR'].astype(str) + '-' + seasonal['MONTH'].astype(str).str.zfill(2)
pivot = seasonal.pivot_table(index='PERIOD', columns='DISAGMOT', values='SHIPWT', aggfunc='sum')
pivot.plot(title='Seasonal Freight Movement by Mode')
plt.ylabel('Total Weight Shipped')
plt.xlabel('Period (Year-Month)')
plt.xticks(rotation=90)
plt.legend(title='Mode (DISAGMOT)', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

The following plot shows seasonal trends in total freight costs by mode.

In [ ]:
# Analyze seasonal trends in freight costs by mode
cost_seasonal = dot2.groupby(['YEAR', 'MONTH', 'DISAGMOT'])['FREIGHT_CHARGES'].sum().reset_index()
cost_seasonal['PERIOD'] = cost_seasonal['YEAR'].astype(str) + '-' + cost_seasonal['MONTH'].astype(str).str.zfill(2)
cost_pivot = cost_seasonal.pivot_table(index='PERIOD', columns='DISAGMOT', values='FREIGHT_CHARGES', aggfunc='sum')
cost_pivot.plot(title='Seasonal Freight Costs by Mode')
plt.ylabel('Total Freight Charges')
plt.xlabel('Period (Year-Month)')
plt.xticks(rotation=90)
plt.legend(title='Mode (DISAGMOT)', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
# Aggregate total freight movement and value by period
movement = dot1.groupby(['YEAR', 'MONTH'])[['SHIPWT', 'VALUE']].sum().reset_index()
movement['PERIOD'] = movement['YEAR'].astype(str) + '-' + movement['MONTH'].astype(str).zfill(2)
movement.set_index('PERIOD', inplace=True)

# Visualize freight movement and value over time
movement[['SHIPWT', 'VALUE']].plot(secondary_y='VALUE', title='Freight Movement vs. Trade Value Over Time')
plt.xlabel('Period (Year-Month)')
plt.tight_layout()
plt.show()

# Display correlation between freight movement and trade value
print("Correlation between SHIPWT and VALUE:")
display(movement[['SHIPWT', 'VALUE']].corr())

In [ ]:
# Summarize total weight shipped by port/district
port_util = dot1.groupby('DEPE')['SHIPWT'].sum().sort_values(ascending=False)
print("Top 10 utilized ports/districts (by total weight):")
display(port_util.head(10))

# Visualize top 20 ports/districts by total weight
port_util.head(20).plot(kind='bar', title='Top 20 Ports/Districts by Total Weight')
plt.ylabel('Total Weight Shipped')
plt.xlabel('Port/District Code (DEPE)')
plt.tight_layout()
plt.show()

In [ ]:
# Analyze containerization by mode (total weight)
if 'CONTCODE' in dot1.columns:
    cont_eff = dot1.groupby(['CONTCODE', 'DISAGMOT'])['SHIPWT'].sum().unstack(fill_value=0)
    cont_eff.plot(kind='bar', stacked=True, title='Containerization by Mode (Total Weight)')
    plt.ylabel('Total Weight Shipped')
    plt.xlabel('Container Code (CONTCODE)')
    plt.legend(title='Mode (DISAGMOT)', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()
else:
    print("CONTCODE not found in dot1 for containerization analysis.")

# Analyze cost per weight by containerization and mode
if 'CONTCODE' in dot2.columns:
    dot2['COST_PER_WEIGHT'] = dot2['FREIGHT_CHARGES'] / dot2['SHIPWT'].replace(0, np.nan)
    cont_cost = dot2.groupby(['CONTCODE', 'DISAGMOT'])['COST_PER_WEIGHT'].mean().unstack(fill_value=0)
    cont_cost.plot(kind='bar', stacked=True, title='Containerization Efficiency by Mode (Cost per Weight)')
    plt.ylabel('Average Cost per Weight')
    plt.xlabel('Container Code (CONTCODE)')
    plt.legend(title='Mode (DISAGMOT)', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()
else:
    print("CONTCODE not found in dot2 for containerization efficiency analysis.")

In [ ]:
display(route_ineff.head(3))

### Five Actionable Recommendations

1. Target the top 3 inefficient mode-route combinations for process improvement and cost reduction.
2. Promote containerization in modes/routes where it significantly improves efficiency.
3. Invest in infrastructure upgrades at the top 5 most utilized ports/districts to prevent bottlenecks.
4. Encourage modal shifts in regions where environmental impact is highest, favoring lower-emission modes.
5. Monitor seasonal peaks and adjust capacity planning to reduce congestion and improve service reliability.

---

## 14. Methodology

- Data was loaded and standardized from three combined CSVs.
- Each analytical question was mapped to the most relevant dataset and columns.
- Visualizations were created using matplotlib and seaborn.
- All code and steps are documented for reproducibility.

---

## 15. Limitations

- Economic indicators (e.g., GDP, unemployment) are not present in the data, so only internal correlations are shown for economic disruption impact.
- If any code cell above prints a message about missing columns, that analysis could not be completed due to data limitations.

---

*End of notebook.*